In [1]:
import glob

import numpy

import pynsx as nsx

In [2]:
####################################
# IMPORT DATA
####################################

# Create an experiment for D19 data
expt = nsx.Experiment('test','D19')

# Get the D19 diffractometer
diffractometer = expt.getDiffractometer()

# Read the D19 datas
data_files = glob.glob("/home/pellegrini/data/d19/reference_data/thom/y-corrected-data/*.h5")

# Loop over the data files
numors = nsx.vector_data()
for f in data_files[0:5]:
    reader = nsx.HDF5DataReader(f,diffractometer)
    # Discard the data with a low number of frames (unlikely to be "production" data)
    if (reader.getNFrames() < 900):
        continue

    # Build the dataset
    data = nsx.DataSet(reader,diffractometer)

    # Add the data to the experiment
    expt.addData(data)

    numors.push_back(data)

In [3]:
####################################
# PEAK FIND
####################################

# Create a kernel for convoluting images prior searching for peak
kernel = nsx.AnnularKernel(data.getNRows(),data.getNCols())
kernel_image = kernel.getKernel()
convolver = nsx.Convolver()
convolver.setKernel(kernel_image)

# Setup the peak finder algorithm
finder = nsx.PeakFinder()
finder.setConvolver(convolver)
finder.setMinComponents(30)
finder.setMaxComponents(10000)
finder.setSearchConfidence(0.67)
finder.setIntegrationConfidence(0.98)
finder.setThresholdType(1)
finder.setThresholdValue(0.4)
finder.setKernel(kernel)

# Search for peaks
peaks = finder.find(numors)

In [4]:
####################################
# AUTO-INDEXING
####################################

indexer = nsx.AutoIndexer(nsx.ProgressHandler())

for peak in peaks:
    if peak.isSelected() and not peak.isMasked():
        indexer.addPeak(peak)

params = nsx.IndexerParameters()
indexer.autoIndex(params)

solutions = indexer.getSolutions()

cell_parameters = []

for idx, (uc,score) in enumerate(solutions):

    a     = uc.character().a
    b     = uc.character().b
    c     = uc.character().c
    alpha = numpy.rad2deg(uc.character().alpha)
    beta  = numpy.rad2deg(uc.character().beta)
    gamma = numpy.rad2deg(uc.character().gamma)

    cell_parameters.append((idx,a,b,c,alpha,beta,gamma,score))

    print("idx: {0:3d} --- a = {1:6.2f}  b = {2:6.2f}  c = {3:6.2f}  alpha = {4:6.2f}  beta = {5:6.2f}  gamma = {6:6.2f} --- % indexed = {7:5.2f}".format(*(cell_parameters[-1])))


idx:   0 --- a =   5.58  b =  16.59  c =  17.05  alpha =  71.28  beta =  83.84  gamma =  89.74 --- % indexed = 99.41
idx:   1 --- a =   5.61  b =  16.50  c =  17.16  alpha =  71.33  beta =  83.42  gamma =  89.72 --- % indexed = 99.41
idx:   2 --- a =   5.61  b =  16.50  c =  17.16  alpha =  71.33  beta =  83.42  gamma =  89.72 --- % indexed = 99.41
idx:   3 --- a =   5.58  b =   5.53  c =  16.15  alpha =  90.00  beta =  96.42  gamma =  90.00 --- % indexed = 97.63
idx:   4 --- a =   5.58  b =   5.53  c =  16.15  alpha =  90.00  beta =  96.42  gamma =  90.00 --- % indexed = 97.63
idx:   5 --- a =   5.58  b =   5.53  c =  16.15  alpha =  90.00  beta =  96.42  gamma =  90.00 --- % indexed = 97.63
idx:   6 --- a =   5.58  b =   5.53  c =  16.15  alpha =  90.00  beta =  96.42  gamma =  90.00 --- % indexed = 97.63
idx:   7 --- a =   5.58  b =   5.53  c =  16.15  alpha =  90.00  beta =  96.42  gamma =  90.00 --- % indexed = 97.63
idx:   8 --- a =   5.58  b =  11.06  c =  17.05  alpha =  71.29 

In [5]:
selected_cell = int(input("Enter the selected unit cell id:"))
assert(selected_cell >= 0 and selected_cell < len(solutions))

Enter the selected unit cell id:5


In [13]:
uc = nsx.UnitCell(solutions[selected_cell][0])
sample = diffractometer.getSample()
sample.addUnitCell(uc)

for peak in peaks:
    if peak.isSelected() and not peak.isMasked():
        peak.addUnitCell(uc, True)

In [18]:
####################################
# SPACE-GROUP
####################################

compatible_space_groups = uc.compatibleSpaceGroups()

hkls = nsx.vectorRowVector3d()
for peak in peaks:
    if peak.isSelected() and not peak.isMasked():
        hkls.append(numpy.array(uc.getIntegerMillerIndices(peak.getQ()),dtype=numpy.float))

space_groups = []
for idx,symbol in enumerate(compatible_space_groups):
    sg = nsx.SpaceGroup(symbol)
    space_groups.append((idx,symbol,100.0*(1.0-sg.fractionExtinct(hkls))))
    print("idx: {0:3d} --- symbol = {1:10s}  --- % non-extincted peaks = {2:6.2f}".format(*(space_groups[-1])))

idx:   0 --- symbol = P 2         --- % non-extincted peaks = 100.00
idx:   1 --- symbol = P 21        --- % non-extincted peaks = 100.00
idx:   2 --- symbol = P m         --- % non-extincted peaks = 100.00
idx:   3 --- symbol = P c         --- % non-extincted peaks = 100.00
idx:   4 --- symbol = P 2/m       --- % non-extincted peaks = 100.00
idx:   5 --- symbol = P 21/m      --- % non-extincted peaks = 100.00
idx:   6 --- symbol = P 2/c       --- % non-extincted peaks = 100.00
idx:   7 --- symbol = P 21/c      --- % non-extincted peaks = 100.00


In [19]:
selected_space_group = int(input("Enter the selected space group id:"))
assert(selected_space_group >= 0 and selected_space_group < len(space_groups))

Enter the selected space group id:1


In [20]:
uc.setSpaceGroup(space_groups[selected_space_group][1])

In [21]:
print(uc.getSpaceGroup())

P 21
